In [ ]:
import glob
import os

os.environ["NEMO_CACHE_DIR"] = "/shared/A_track/"
glob.glob("/shared/KASR/nemo/nemo_experiments/default/checkpoints/epoch=*.ckpt")


In [ ]:
%cd /shared/KASR/nemo

In [ ]:
import utils

# import KASR.nemo.utils as utils
device = utils.get_device_safe_threading()

In [ ]:
import glob
epoch = "14"
checkpoint_paths = glob.glob(f"/shared/KASR/nemo/nemo_experiments/default/checkpoints/epoch={epoch}*.ckpt")
checkpoint_paths

In [ ]:
from nemo.collections.asr.models import EncDecCTCModelBPE
import torch



checkpoint_path = checkpoint_paths[-1]

# Path to save the exported .nemo file
nemo_path = f"/shared/KASR/nemo/nemo_experiments/default/finetuned_epoch={epoch}_model.nemo"

# Load model from .ckpt
asr_model = EncDecCTCModelBPE.load_from_checkpoint(checkpoint_path, map_location="cuda" if torch.cuda.is_available() else "cpu")

asr_model.decoding.strategy = 'greedy_batch'

# Export to .nemo file
asr_model.save_to(nemo_path)

print(f"Model exported to {nemo_path}")


In [ ]:
raw_path = "/shared/track_a_audio_files"
train_json_path = "/shared/A_track/test.json"

import pandas as pd
import os

test_df = pd.read_json(train_json_path).T


test_df["file_path"] = "processed/"+ test_df["audio_path"] +".mel.pt"
test_df['audio'] = test_df['audio_path'].apply(lambda x: os.path.join(raw_path, x.replace("audio/", "") +'.wav'))
test_df

In [ ]:
from nemo.collections.asr.models import EncDecCTCModelBPE


asr_model = EncDecCTCModelBPE.restore_from(restore_path=nemo_path, map_location=device, )



In [ ]:
# wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/asr_language_modeling/ngram_lm/install_beamsearch_decoders.sh
# chmod +x install_beamsearch_decoders.sh

# !mkdir -p $HOME/nemo_decoders

# # Run the install script using your user directory (no sudo needed)
# !bash install_beamsearch_decoders.sh $HOME/nemo_decoders

In [ ]:
# !pip install kenlm flashlight-text pyctcdecode


In [ ]:

# Optionally, set decoding strategy for faster inference
# asr_model.decoding.strategy = 'greedy_batch'
import copy
decoding_config = copy.deepcopy(asr_model.cfg.decoding)
decoding_config.strategy = "beam"
decoding_config.beam.beam_size = 8  # You can adjust this number
asr_model.change_decoding_strategy(decoding_config)

# Transcribe
results = asr_model.transcribe(test_df['audio'].to_list(),
    batch_size=4, # Adjust batch size based on your GPU memory

)

results_list = []
# Print results
for result in results:
    print(result.text)
    results_list.append(result.text)

test_df['transcription'] = results_list
test_df['id'] = test_df.index
test_df[["id", "transcription"]].to_csv(f"/shared/A_track/submission_epoch_{epoch}.csv", index=False)



In [ ]:
# Optionally, set decoding strategy for faster inference
# asr_model.decoding.strategy = 'greedy_batch'

# Transcribe
results = asr_model.transcribe(test_df['audio'].to_list())

results_list = []
# Print results
for result in results:
    print(result.text)
    results_list.append(result.text)

test_df['transcription'] = results_list
test_df['id'] = test_df.index
test_df[["id", "transcription"]].to_csv(f"/shared/A_track/submission_epoch_{epoch}.csv", index=False)


In [ ]:
# Save the submission file path
submission_path = f"/shared/A_track/submission_epoch_{epoch}.csv"

# Make sure kaggle is installed and API credentials are set up in ~/.kaggle/kaggle.json

# Example: Upload to a Kaggle competition (replace 'your-competition-name' with the actual competition name)
!kaggle competitions submit -c kinyarwanda-automatic-speech-recognition-track-a -f "{submission_path}" -m "Submission for epoch {epoch} Automated from the script"

In [ ]:
# import tarfile

# with tarfile.open("/shared/track_b_audio.tar.gz", "w:gz") as tar:
#     # Add audio files from track_b_audio_files
#     for fname in test_df["audio"]:
#         tar.add(fname)